In [2]:
import sqlite3
import pandas as pd

#### **Merging and Joining**

1. **Inner Join on Chinook Database**
   - Load the `chinook.db` database.
   - Perform an inner join between the `customers` and `invoices` tables on the `CustomerId` column.
   - Find the total number of invoices for each customer.


In [5]:
# Load the chinook.db database
conn = sqlite3.connect('../data/chinook.db')

# Perform an inner join between the customers and invoices tables on the CustomerId column
query = '''
SELECT customers.CustomerId, customers.FirstName, customers.LastName, COUNT(invoices.InvoiceId) as TotalInvoices
FROM customers
INNER JOIN invoices ON customers.CustomerId = invoices.CustomerId
GROUP BY customers.CustomerId
'''

# Execute the query and load the data into a DataFrame
df = pd.read_sql_query(query, conn)

# Close the database connection
conn.close()

# Display the DataFrame
print(df)

    CustomerId  FirstName      LastName  TotalInvoices
0            1       Luís     Gonçalves              7
1            2     Leonie        Köhler              7
2            3   François      Tremblay              7
3            4      Bjørn        Hansen              7
4            5  František   Wichterlová              7
5            6     Helena          Holý              7
6            7     Astrid        Gruber              7
7            8       Daan       Peeters              7
8            9       Kara       Nielsen              7
9           10    Eduardo       Martins              7
10          11  Alexandre         Rocha              7
11          12    Roberto       Almeida              7
12          13   Fernanda         Ramos              7
13          14       Mark       Philips              7
14          15   Jennifer      Peterson              7
15          16      Frank        Harris              7
16          17       Jack         Smith              7
17        

2. **Outer Join on Movie Data**
   - Load the `movie.csv` file.
   - Create two smaller DataFrames:
     - One with only `director_name` and `color`.
     - Another with `director_name` and `num_critic_for_reviews`.
   - Perform a left join and then a full outer join on `director_name`.
   - Count how many rows are in the resulting DataFrames for each join type.

In [15]:
# Load the movie.csv file
movie_df = pd.read_csv('../data/movie.csv')

# Create two smaller DataFrames
df1 = movie_df[['director_name', 'color']]
df2 = movie_df[['director_name', 'num_critic_for_reviews']]

# Perform a left join on director_name
left_join_df = pd.merge(df1, df2, on='director_name', how='left')

# Perform a full outer join on director_name
outer_join_df = pd.merge(df1, df2, on='director_name', how='outer')

# Count the number of rows in the resulting DataFrames for each join type
left_join_count = left_join_df.shape[0]
outer_join_count = outer_join_df.shape[0]

print(f'Number of rows in left join DataFrame: {left_join_count}')
print(f'Number of rows in full outer join DataFrame: {outer_join_count}')

Number of rows in left join DataFrame: 30300
Number of rows in full outer join DataFrame: 30300


#### **Grouping and Aggregating**
1. **Grouped Aggregations on Titanic**
   - Group passengers by `Pclass` and calculate the following:
     - Average age.
     - Total fare.
     - Count of passengers.
   - Save the results to a new DataFrame.

In [6]:
# Load the Titanic dataset
titanic_df = pd.read_excel('../data/titanic.xlsx')

# Group passengers by Pclass and calculate the average age, total fare, and count of passengers
grouped_df = titanic_df.groupby('Pclass').agg({
    'Age': 'mean',
    'Fare': 'sum',
    'PassengerId': 'count'
}).rename(columns={'Age': 'Average Age', 'Fare': 'Total Fare', 'PassengerId': 'Passenger Count'})

# Display the resulting DataFrame
print(grouped_df)

        Average Age  Total Fare  Passenger Count
Pclass                                          
1         38.233441  18177.4125              216
2         29.877630   3801.8417              184
3         25.140620   6714.6951              491


2. **Multi-level Grouping on Movie Data**
   - Group the movies by `color` and `director_name`.
   - Find:
     - Total `num_critic_for_reviews` for each group.
     - Average `duration` for each group.

In [7]:
# Group the movies by color and director_name
grouped_movie_df = movie_df.groupby(['color', 'director_name']).agg({
    'num_critic_for_reviews': 'sum',
    'duration': 'mean'
}).rename(columns={'num_critic_for_reviews': 'Total Critic Reviews', 'duration': 'Average Duration'})

# Display the resulting DataFrame
print(grouped_movie_df)

                                    Total Critic Reviews  Average Duration
color           director_name                                             
Black and White Akira Kurosawa                     153.0             202.0
                Aleksey German                     121.0             177.0
                Alex Garland                       489.0             108.0
                Alexander Payne                    433.0             115.0
                Alfred Hitchcock                   434.0             119.0
...                                                  ...               ...
Color           Zoran Lisinac                       17.0             108.0
                Álex de la Iglesia                  71.0             104.0
                Émile Gaudreault                    67.0              92.0
                Éric Tessier                         9.0              99.0
                Étienne Faure                        9.0              98.0

[2490 rows x 2 columns]


3. **Nested Grouping on Flights**
   - Group flights by `Year` and `Month` and calculate:
     - Total number of flights.
     - Average arrival delay (`ArrDelay`).
     - Maximum departure delay (`DepDelay`).

In [20]:
# Load the flights dataset
flights_df = pd.read_parquet('../data/mtcars.parquet')

# Check if required columns exist
required_columns = {'model', 'mpg', 'cyl', 'disp'}
missing_columns = required_columns - set(flights_df.columns)

grouped_flights_df = flights_df.groupby(['model', 'mpg']).agg({
    'model': 'count',
    'mpg': 'mean',
}).rename(columns={'model': 'Total Flights', 'mpg': 'Average Arrival Delay'})

# Display the resulting DataFrame
print(grouped_flights_df)

                          Total Flights  Average Arrival Delay
model               mpg                                       
AMC Javelin         15.2              1                   15.2
Cadillac Fleetwood  10.4              1                   10.4
Camaro Z28          13.3              1                   13.3
Chrysler Imperial   14.7              1                   14.7
Datsun 710          22.8              1                   22.8
Dodge Challenger    15.5              1                   15.5
Duster 360          14.3              1                   14.3
Ferrari Dino        19.7              1                   19.7
Fiat 128            32.4              1                   32.4
Fiat X1-9           27.3              1                   27.3
Ford Pantera L      15.8              1                   15.8
Honda Civic         30.4              1                   30.4
Hornet 4 Drive      21.4              1                   21.4
Hornet Sportabout   18.7              1                

#### **Applying Functions**
1. **Apply a Custom Function on Titanic**
   - Write a function to classify passengers as `Child` (age < 18) or `Adult`.
   - Use `apply` to create a new column, `Age_Group`, with these values.


In [7]:
# Define the custom function to classify passengers
def classify_age(age):
    if age < 18:
        return 'Child'
    else:
        return 'Adult'

# Apply the function to create a new column 'Age_Group'
titanic_df['Age_Group'] = titanic_df['Age'].apply(classify_age)

# Display the DataFrame with the new column
print(titanic_df[['Age', 'Age_Group']])

      Age Age_Group
0    22.0     Adult
1    38.0     Adult
2    26.0     Adult
3    35.0     Adult
4    35.0     Adult
..    ...       ...
886  27.0     Adult
887  19.0     Adult
888   NaN     Adult
889  26.0     Adult
890  32.0     Adult

[891 rows x 2 columns]



2. **Normalize Employee Salaries**
   - Load the `employee.csv` file.
   - Normalize the salaries within each department.

In [13]:
# Load the employee.csv file
employee_df = pd.read_csv('../data/emplayee.csv')

# Define a function to normalize the salaries
def normalize_salary(salary, department):
    dept_salaries = employee_df[employee_df['DEPARTMENT'] == department]['BASE_SALARY']
    return (salary - dept_salaries.mean()) / dept_salaries.std()

# Apply the function to normalize the salaries within each department
employee_df['NORMALIZED_SALARY'] = employee_df.apply(lambda row: normalize_salary(row['BASE_SALARY'], row['DEPARTMENT']), axis=1)

# Display the DataFrame with the new column
print(employee_df[['DEPARTMENT', 'BASE_SALARY', 'NORMALIZED_SALARY']])

                         DEPARTMENT  BASE_SALARY  NORMALIZED_SALARY
0       Municipal Courts Department     121862.0           2.568478
1                           Library      26125.0          -0.962884
2     Houston Police Department-HPD      45279.0          -0.892524
3     Houston Fire Department (HFD)      63166.0           0.214332
4       General Services Department      56347.0           0.224533
...                             ...          ...                ...
1995  Houston Police Department-HPD      43443.0          -1.001846
1996  Houston Fire Department (HFD)      66523.0           0.412156
1997  Houston Police Department-HPD      43443.0          -1.001846
1998  Houston Police Department-HPD      55461.0          -0.286254
1999  Houston Fire Department (HFD)      51194.0          -0.491163

[2000 rows x 3 columns]


3. **Custom Function on Movies**
   - Write a function that returns `Short`, `Medium`, or `Long` based on the duration of a movie:
     - `Short`: Less than 60 minutes.
     - `Medium`: Between 60 and 120 minutes.
     - `Long`: More than 120 minutes.
   - Apply this function to classify movies in the `movie.csv` dataset.

In [17]:
# Define the custom function to classify movies based on duration
def classify_duration(duration):
    if duration < 60:
        return 'Short'
    elif 60 <= duration <= 120:
        return 'Medium'
    else:
        return 'Long'

# Apply the function to create a new column 'Duration_Category'
movie_df['Duration_Category'] = movie_df['duration'].apply(classify_duration)

# Display the DataFrame with the new column
print(movie_df[['duration', 'Duration_Category']])

      duration Duration_Category
0        178.0              Long
1        169.0              Long
2        148.0              Long
3        164.0              Long
4          NaN              Long
...        ...               ...
4911      87.0            Medium
4912      43.0             Short
4913      76.0            Medium
4914     100.0            Medium
4915      90.0            Medium

[4916 rows x 2 columns]


#### **Using `pipe`**
1. **Pipeline on Titanic**
   - Create a pipeline to:
     - Filter passengers who survived (`Survived == 1`).
     - Fill missing `Age` values with the mean.
     - Create a new column, `Fare_Per_Age`, by dividing `Fare` by `Age`.


In [18]:
# Define a function to filter passengers who survived
def filter_survived(df):
    return df[df['Survived'] == 1]

# Define a function to fill missing Age values with the mean
def fill_missing_age(df):
    df['Age'].fillna(df['Age'].mean(), inplace=True)
    return df

# Define a function to create a new column Fare_Per_Age
def create_fare_per_age(df):
    df['Fare_Per_Age'] = df['Fare'] / df['Age']
    return df

# Create the pipeline
titanic_pipeline_df = (titanic_df
                       .pipe(filter_survived)
                       .pipe(fill_missing_age)
                       .pipe(create_fare_per_age))

# Display the resulting DataFrame
print(titanic_pipeline_df[['Survived', 'Age', 'Fare', 'Fare_Per_Age']])

     Survived   Age     Fare  Fare_Per_Age
1           1  38.0  71.2833      1.875876
2           1  26.0   7.9250      0.304808
3           1  35.0  53.1000      1.517143
8           1  27.0  11.1333      0.412344
9           1  14.0  30.0708      2.147914
..        ...   ...      ...           ...
875         1  15.0   7.2250      0.481667
879         1  56.0  83.1583      1.484970
880         1  25.0  26.0000      1.040000
887         1  19.0  30.0000      1.578947
889         1  26.0  30.0000      1.153846

[342 rows x 4 columns]


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_17936\1152674106.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(df['Age'].mean(), inplace=True)
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_17936\1152674106.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Age'].fillna(df['Age'].mean(), inplace=True)
C:\Users\ADMIN\AppDat

2. **Pipeline on Flights**
   - Create a pipeline to:
     - Filter flights with a departure delay greater than 30 minutes.
     - Add a column `Delay_Per_Hour` by dividing the delay by the scheduled flight duration.

In [22]:
# Define a function to filter flights with a departure delay greater than 30 minutes
def filter_departure_delay(df):
    return df[df['mpg'] > 30]

# Define a function to add a column Delay_Per_Hour by dividing the delay by the scheduled flight duration
def add_delay_per_hour(df):
    df['ratio'] = df['mpg'] / df['cyl']
    return df

# Create the pipeline
flights_pipeline_df = (flights_df
                       .pipe(filter_departure_delay)
                       .pipe(add_delay_per_hour))

# Display the resulting DataFrame
print(flights_pipeline_df[['model', 'mpg', 'cyl', 'ratio']])

             model   mpg  cyl  ratio
17        Fiat 128  32.4    4  8.100
18     Honda Civic  30.4    4  7.600
19  Toyota Corolla  33.9    4  8.475
27    Lotus Europa  30.4    4  7.600


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_17936\3032237569.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ratio'] = df['mpg'] / df['cyl']
